# Validation results

In this notebook we examine how well Oxford Nanopore SNP calling agrees with SNP calling from Illumina data.

The parameters we use are:
    * nanopolish: 5-mer alignment, 6-mer variant calling, reverse HMM fix

In [1]:
ROOTDIR="/mnt/ebola/ebov"
import os
os.environ['PATH'] = os.getenv('PATH') + ':' + ROOTDIR + '/scripts'

In [2]:
%cd $ROOTDIR

/mnt/ebola/ebov


In [ ]:
%%bash

ROOTDIR="/mnt/ebola/ebov"
cd $ROOTDIR
export PATH=`pwd`/bin/bwa:$PATH
export PATH=$PATH:`pwd`/scripts
export PATH=$PATH:`pwd`/nanopore-scripts
export PATH=$PATH:`pwd`/bin/nanopolish-6mer
export PATH=$PATH:`pwd`/bin/samtools
export PATH=$PATH:`pwd`/bin/marginAlign

mkdir -p 180genomes-validation
cd 180genomes-validation

echo "../models/6mer/ont_complement.pop1.model" > offset_models.fofn
echo "../models/6mer/ont_complement.pop2.model" >> offset_models.fofn
echo "../models/6mer/ont_template.model" >> offset_models.fofn

makecommands.py ../metadata/metadata.db 180_Genomes align_defaultkmer_margin.sh | parallel -j16

In [10]:
## After the raw files are produced, show statistics.

%cd 180genomes-validation

[Errno 2] No such file or directory: '180genomes-validation'
/mnt/ebola/ebov/180genomes-validation


In [11]:
fh = open("validation_files.txt", "w")
fh.write("""076534_180Genomes_11rx	np-var6mer-aln5mer	../refs/EM_079517_EM_076534.mutations.txt	np_EM_079517_076534_180Genomes_11rx_hq.vcf
076533_180Genomes_11rx	np-var6mer-aln5mer	../refs/EM_079517_EM_076533.mutations.txt	np_EM_079517_076533_180Genomes_11rx_hq.vcf
076383_180Genomes_11rx	np-var6mer-aln5mer	../refs/EM_079517_EM_076383.mutations.txt	np_EM_079517_076383_180Genomes_11rx_hq.vcf
078416_180Genomes_11rx	np-var6mer-aln5mer	../refs/EM_079517_EM_078416.mutations.txt	np_EM_079517_078416_180Genomes_11rx_hq.vcf
076769_180Genomes_19Rx	np-var6mer-aln5mer	../refs/EM_079517_EM_076769.mutations.txt	np_EM_079517_076769_180Genomes_19Rx_hq.vcf
""")
fh.close()

In [12]:
!../scripts/intersection_vcf_stats.py validation_files.txt | cut -f 1,2,5,6,7,8,9,10 | column -t 

sample                  tag                 total_calls  mutations  TP    FP   FN   TPR
076534_180Genomes_11rx  np-var6mer-aln5mer  14           18         14.0  0.0  4.0  0.777777777778
076533_180Genomes_11rx  np-var6mer-aln5mer  20           20         19.0  1.0  1.0  0.95
076383_180Genomes_11rx  np-var6mer-aln5mer  17           18         16.0  1.0  2.0  0.888888888889
078416_180Genomes_11rx  np-var6mer-aln5mer  18           18         17.0  1.0  1.0  0.944444444444
076769_180Genomes_19Rx  np-var6mer-aln5mer  19           19         19.0  0.0  0.0  1.0


In [ ]:
!../scripts/intersection_vcf_interrogate.py validation_files.txt > validation_stats.txt

In [ ]:
!head -10 validation_stats.txt

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

library(ggplot2)

stats=read.table("validation_stats.txt", header=T)
ggplot(stats, aes(x=supportfraction, fill=state)) + geom_histogram() + facet_wrap(~tag)

## Remove low depth positions from plot (depth >= 30)

In [ ]:
%%R

library(ggplot2)

stats=read.table("validation_stats.txt", header=T)
ggplot(subset(stats, supportingreads>50), aes(x=supportfraction, fill=state)) + geom_histogram(breaks=c()) + facet_wrap(~tag)

In [ ]:
%%R

library(ggplot2)

stats=read.table("validation_stats.txt", header=T)
ggplot(subset(stats, supportingreads>50), aes(x=qual, fill=state)) + geom_histogram(breaks=c()) + xlim(0,2000) + facet_wrap(~tag)

## Filter the files as per the study (2d depth>= 25, 1d depth >=50, LL > 200, primer sequences masked)

In [14]:
!makecommands.py ../metadata/metadata.db 180_Genomes consensus | parallel -j16

+ ref_prefix=EM_079517
+ sample=076534_180Genomes_11rx
+ poretools_dir=076534_180Genomes_11rx
+ sample_tag=076534_180Genomes_11rx_hq
+ second_batch=na
+ vcftagprimersites.py all np_EM_079517_076534_180Genomes_11rx_hq.vcf
+ vcffilter.py 076534_180Genomes_11rx_hq_EM_079517_np_primer.tagged.vcf
Output 14 records
+ vcffilterqual.py 076534_180Genomes_11rx_hq_EM_079517_np_primer.tagged.vcf
Output 14 records
+ ref_prefix=EM_079517_mut30_2
+ sample=076534_180Genomes_11rx
+ poretools_dir=076534_180Genomes_11rx
+ sample_tag=076534_180Genomes_11rx_hq
+ second_batch=na
+ vcftagprimersites.py all np_EM_079517_mut30_2_076534_180Genomes_11rx_hq.vcf
+ vcffilter.py 076534_180Genomes_11rx_hq_EM_079517_mut30_2_np_primer.tagged.vcf
Output 0 records
+ vcffilterqual.py 076534_180Genomes_11rx_hq_EM_079517_mut30_2_np_primer.tagged.vcf
Output 0 records
+ ref_prefix=EM_079517_mut30_2
+ sample=076383_180Genomes_11rx
+ poretools_dir=076383_180Genomes_11rx
+ sample_tag=076383_180Genomes_11rx_hq
+ second_batch=na
+

In [16]:
fh = open("validation_files_filtered.txt", "w")
fh.write("""076534_180Genomes_11rx	np-6mer-5meralign	../refs/EM_079517_EM_076534.mutations.txt	076534_180Genomes_11rx_hq_EM_079517_np_primer.filtered_qual200.vcf
076533_180Genomes_11rx	np-6mer-5meralign	../refs/EM_079517_EM_076533.mutations.txt	076533_180Genomes_11rx_hq_EM_079517_np_primer.filtered_qual200.vcf
076383_180Genomes_11rx	np-6mer-5meralign	../refs/EM_079517_EM_076383.mutations.txt	076383_180Genomes_11rx_hq_EM_079517_np_primer.filtered_qual200.vcf
078416_180Genomes_11rx	np-6mer-5meralign	../refs/EM_079517_EM_078416.mutations.txt	078416_180Genomes_11rx_hq_EM_079517_np_primer.filtered_qual200.vcf
076769_180Genomes_19Rx	np-6mer-5meralign	../refs/EM_079517_EM_076769.mutations.txt	076769_180Genomes_19Rx_hq_EM_079517_np_primer.filtered_qual200.vcf""")
fh.close()

In [17]:
!../scripts/intersection_vcf_stats.py validation_files_filtered.txt | cut -f 1,2,5,6,7,8,9,10 | column -t 

sample                  tag                total_calls  mutations  TP    FP   FN   TPR
076534_180Genomes_11rx  np-6mer-5meralign  14           18         14.0  0.0  4.0  0.777777777778
076533_180Genomes_11rx  np-6mer-5meralign  19           20         19.0  0.0  1.0  0.95
076383_180Genomes_11rx  np-6mer-5meralign  16           18         16.0  0.0  2.0  0.888888888889
078416_180Genomes_11rx  np-6mer-5meralign  17           18         17.0  0.0  1.0  0.944444444444
076769_180Genomes_19Rx  np-6mer-5meralign  19           19         19.0  0.0  0.0  1.0


In [ ]:
!makecommands.py ../metadata/metadata.db 180_Genomes qual.sh | parallel


In [ ]:
!collect_quals.py ../metadata/metadata.db 180_Genomes > stats.txt

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(ggplot2)
library(grid)
library(plyr)
a=read.table("stats.txt", sep="\t", header=T)
a=subset(a, aln == 'ma')
a$perc = a$matches/(a$matches + a$insertions + a$deletions + a$mismatches)*100
cdat <- ddply(a, "sample", summarise, sample.mean=mean(perc))
cdat
ggplot(a, aes(x=perc)) +
  geom_histogram() +
  scale_x_continuous(breaks=c(60, 70, 80, 90, 100)) +
  geom_vline(data=cdat, aes(xintercept=sample.mean), size=0.2, linetype="dashed") +
  theme_bw(base_size=8) + xlab("Percentage accuracy") +
  facet_wrap(~ sample, ncol=1)
 # theme(strip.text = element_blank(), strip.background = element_blank(), panel.margin.y = unit(-0.5, "lines"))
#ggsave("percs_144_ma.pdf", height=12, width=9)


In [ ]:
!pwd

In [ ]:
%%R
library(ggplot2)
library(grid)
library(plyr)
a=read.table("stats.txt", sep="\t", header=T)
a=subset(a, aln == 'ma')
a$perc = a$matches/(a$matches + a$insertions + a$deletions + a$mismatches)*100
cdat <- ddply(a, "sample", summarise, sample.mean=mean(perc))
cdat
ggplot(a, aes(x=perc)) +
  geom_histogram() +
  scale_x_continuous(breaks=c(60, 70, 80, 90, 100)) +
  geom_vline(data=cdat, aes(xintercept=sample.mean), size=0.2, linetype="dashed") +
  theme_bw(base_size=8) + xlab("Percentage accuracy") +
  facet_wrap(~ sample, ncol=5)
 # theme(strip.text = element_blank(), strip.background = element_blank(), panel.margin.y = unit(-0.5, "lines"))
#ggsave("percs_144_ma.pdf", height=12, width=9)


In [18]:
!make -f ../scripts/filter_clusters.mk METADATA=../metadata/metadata.db SET=180_Genomes PREFIX=180_Genomes TPR=0.0 TPRMAX=1.0

make_stats_file.py ../metadata/metadata.db 180_Genomes  | intersection_vcf_stats.py /dev/stdin | awk '($10>=0.0&&$10<=1.0)' | grep np-new-filter_qual200-50 | cut -f1 | sort | uniq | xargs -L 1 -I '{}' margin_cons.py ../refs/EM_079517.fasta {}_hq_EM_079517_np_primer.tagged.vcf EM_079517_{}_hq_marginalign.sorted.bam all >180_Genomes-tpr0.0.fasta 2>180_Genomes-tpr0.0.stderr
tagfastas.py ../metadata/metadata.db < 180_Genomes-tpr0.0.fasta > 180_Genomes-tpr0.0.tagged.fasta


In [19]:
!grep ">" 180_Genomes-tpr0.0.fasta

>EM_079517_076383_180Genomes_11rx_hq_marginalign.sorted.bam
>EM_079517_076533_180Genomes_11rx_hq_marginalign.sorted.bam
>EM_079517_076534_180Genomes_11rx_hq_marginalign.sorted.bam
>EM_079517_076769_180Genomes_19Rx_hq_marginalign.sorted.bam
>EM_079517_078416_180Genomes_11rx_hq_marginalign.sorted.bam


# Get the 180 genome set

In [20]:
from Bio import Entrez
Entrez.email = "n.j.loman@bham.ac.uk"     # Always tell NCBI who you are
handle = Entrez.esearch(db="nucleotide", term="carroll and hiscox and ebola", retmax=200)
record = Entrez.read(handle)
handle = Entrez.efetch(db="nucleotide", id=record["IdList"], rettype="fasta", retmode="text")
fh = open("180_Genomes_references.fasta", "w")
fh.write(handle.read())

In [ ]:
!echo "\n"  >> 180_Genomes_references.fasta 
!cat 180_Genomes_references.fasta 180_Genomes-tpr0.0.fasta > validate_tree.fasta
!sed --in-place '/^$/d' validate_tree.fasta
!../bin/muscle/muscle3.8.31_i86linux64 -in validate_tree.fasta > validate_tree_aligned.fasta


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

validate_tree 184 seqs, max length 18960, avg  length 18921
00:00:04    29 MB(-2%)  Iter   1  100.00%  K-mer dist pass 1
00:00:04    29 MB(-2%)  Iter   1  100.00%  K-mer dist pass 2


In [23]:
!sed --in-place 's/>.*H.sapiens-wt\//>/' validate_tree_aligned.fasta
!sed --in-place 's/>EM_079517_/>MinION_/' validate_tree_aligned.fasta
!sed --in-place 's/,//' validate_tree_aligned.fasta

In [25]:
!sed --in-place 's/_hq_marginalign.sorted.bam//' validate_tree_aligned.fasta

In [26]:
!grep ">" validate_tree_aligned.fasta

>MinION_076534_180Genomes_11rx
>MinION_076769_180Genomes_19Rx
>MinION_076383_180Genomes_11rx
>MinION_078416_180Genomes_11rx
>MinION_076533_180Genomes_11rx
>GIN/2014/Makona-EM_076615 partial genome
>GIN/2014/Makona-EM_079388 partial genome
>GIN/2014/Makona-EM_078415 partial genome
>GIN/2014/Makona-EM_076138 partial genome
>GIN/2014/Makona-EM_076770 partial genome
>GIN/2014/Makona-EM_080076 partial genome
>GIN/2014/Makona-EM_074354 partial genome
>GIN/2014/Makona-EM_078763 partial genome
>LBR/2014/Makona-EM_080223 partial genome
>GIN/2014/Makona-EM_000321 partial genome
>GIN/2014/Makona-EM_000500 partial genome
>GIN/2015/Makona-EM_004589 partial genome
>GIN/2014/Makona-EM_079578 partial genome
>GIN/2014/Makona-EM_079587 partial genome
>GIN/2014/Makona-EM_079514 partial genome
>GIN/2014/Makona-EM_074335 partial genome
>GIN/2014/Makona-EM_079859 partial genome
>GIN/2014/Makona-EM_079815 partial genome
>GIN/2014/Makona-EM_079750 partial genome
>GIN/2014/Makona-EM_079775 partial genome
>GIN/

In [28]:
!rm *validate_tree_aligned.raxml
!../bin/standard-RAxML/raxmlHPC-PTHREADS-SSE3 -T 32 -f a -x 12345 -p 12345 -# 100 -m GTRGAMMA -s validate_tree_aligned.fasta -n validate_tree_aligned.raxml

rm: cannot remove ‘*validate_tree_aligned.raxml’: No such file or directory

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



IMPORTANT WARNING: Sequences GIN/2014/Makona-EM_078779 and GIN/2014/Makona-EM_004290 are exactly identical

IMPORTANT WARNING
Found 1 sequence that is exactly identical to other sequences in the alignment.
Normally they should be excluded from the analysis.

Just in case you might need it, an alignment file with 
sequence duplicates removed is printed to file validate_tree_aligned.fasta.reduced


Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 3

This is RAxML Worker Pthread Number: 4

This is RAxML Worker Pthread Number: 2

This is RAxML Worker Pthread Number: 1

This is RAxML Worker Pthread Number: 6

This is RAxML Worker Pthread Number: 7

This is RAxML Worker Pthread Number: 8

This is RAxML Worker

In [ ]:
!cat /mnt/ebola/ebov/analysis/RAxML_bipartitions.validate_tree_aligned.raxml


In [ ]:
!pip install ete2

from ete2 import Tree
t = Tree()
t.populate(20)
t.render(file_name="test.png", w=500)